In [1]:
from qiskit.transpiler import CouplingMap, PassManager
from qiskit.transpiler.passes import SabreLayout, SabreSwap
from qiskit.transpiler.passes import FullAncillaAllocation, EnlargeWithAncilla, ApplyLayout
from utils import file_to_coupling_map, directory_to_circuits

import random
import time
import os
import numpy as np


In [2]:
seed = 42
random.seed(seed)

cmap_names = ["ring_127.txt", "heavy_hex_127.txt", "square_144.txt"]
qc_name = "ghz"
heuristic_list = ["basic", "lookahead", "decay", "depth", "critical"]
qc_list = directory_to_circuits(f"circuits/{qc_name}")
seed_list = [random.randint(0, 100000) for _ in range(len(qc_list))]
pass_trials = 20

# CSV file to store the results
csv_path = f"results/{qc_name}.csv"
# Columns are: "swap_count", "depth", "runtime", "num_qubits", "heuristic", "cmap", "trials", "seed"
with open(csv_path, "w") as f:
    f.write("swap_count,depth,runtime,num_qubits,heuristic,cmap,trials,seed\n")




In [3]:
# Run on all heuristics
for heuristic in heuristic_list:
    print(f"Running {heuristic} heuristic")
    # Run on all coupling maps
    for cmap_name in cmap_names: 
        print(f"    Loading coupling map {cmap_name}")
        cmap = file_to_coupling_map(f"coupling_maps/{cmap_name}")
        # Run on all circuits
        for qc in qc_list:
            num_qubits = qc.num_qubits
            curr_seed = seed_list[qc_list.index(qc)]
            print(f"        Running {qc.name} on {cmap_name} with seed {curr_seed}")
            # Create the routing pass
            rp = SabreSwap(coupling_map=cmap, heuristic=heuristic, seed=curr_seed, trials=pass_trials)
            lp = SabreLayout(coupling_map=cmap, routing_pass=rp, seed=curr_seed)
            # Create the pass manager
            pm = PassManager([lp, FullAncillaAllocation(cmap), EnlargeWithAncilla(), ApplyLayout(), rp])
            # Execute the pass manager
            start = time.time()
            qc_tr = pm.run(qc)
            end = time.time()
            # Collect the metrics for this trial
            runtime = end - start
            swap_count = qc_tr.count_ops().get("swap", 0)
            depth = qc_tr.depth(lambda x: x.operation.num_qubits == 2)

            # Write the results to the CSV file
            # Columns are: "swap_count", "depth", "runtime", "num_qubits", "heuristic", "cmap", "trials", "seed"
            with open(csv_path, "a") as f:
                f.write(f"{swap_count},{depth},{runtime},{num_qubits},{heuristic},{cmap_name},{pass_trials},{curr_seed}\n")
            print(f"            Swap count: {swap_count}, Depth: {depth}, Runtime: {runtime}")

    

Running basic heuristic
    Loading coupling map ring_127.txt
        Running ghz_10.qasm on ring_127.txt with seed 83810
            Swap count: 19, Depth: 22, Runtime: 0.04015088081359863
        Running ghz_29.qasm on ring_127.txt with seed 14592
            Swap count: 99, Depth: 87, Runtime: 0.07198405265808105
        Running ghz_49.qasm on ring_127.txt with seed 3278
            Swap count: 294, Depth: 212, Runtime: 0.08799886703491211
        Running ghz_68.qasm on ring_127.txt with seed 97196
            Swap count: 594, Depth: 380, Runtime: 0.10328388214111328
        Running ghz_88.qasm on ring_127.txt with seed 36048
            Swap count: 658, Depth: 460, Runtime: 0.11690402030944824
        Running ghz_107.qasm on ring_127.txt with seed 32098
            Swap count: 660, Depth: 460, Runtime: 0.12312984466552734
        Running ghz_127.qasm on ring_127.txt with seed 29256
            Swap count: 1355, Depth: 841, Runtime: 0.16850781440734863
    Loading coupling map heavy